In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


In [3]:
df_mal = pd.read_csv("C:\\Users\\amogh\\OneDrive\\Documents\\GitHub\\InnovationProject\\data\\raw\\malware_dataset.csv")
df_benign= pd.read_csv("C:\\Users\\amogh\\OneDrive\\Documents\\GitHub\\InnovationProject\\data\\raw\\benign.csv")

In [ ]:
combined = pd.concat

: 